# Normalize Cell Health Labels

**Gregory Way, 2019**

In [1]:
import os
import numpy as np
from scipy.stats import median_absolute_deviation
import pandas as pd

from pycytominer import write_gct

In [2]:
%matplotlib inline

In [3]:
# Function to scale cell health target variables
def scale(x):
    x_median = np.nanmedian(x)
    x_std = np.nanstd(x)
    x_scale = (x - x_median) / x_std
    return x_scale

In [4]:
file = os.path.join("data", "labels", "cell_health_labels.tsv")
label_df = pd.read_csv(file, sep='\t')

print(label_df.shape)
label_df.head(2)

(2302, 75)


,cell_id,guide,plate_name,well_col,well_row,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,ES2,AKT1-1,Plate 1,3,A,0.04287,0.007976,0.003988,1003,1.777,...,1465.0,0.02710,0.01190,0.01520,1.640,0.03173,0.01652,0.9683,NaN,NaN
1,ES2,AKT1-1,Plate 1,22,O,0.02635,0.005988,0.005988,835,1.582,...,1575.0,0.03169,0.01463,0.01706,1.405,0.03961,0.02255,0.9598,279.6,2083.0


In [5]:
# Some infinite values are present, replace them with NA
label_df = label_df.replace([np.inf, -np.inf], np.nan)

# Apply normalization by plate
normalized_label_df = (
    label_df
    .drop(["guide", "well_col", "well_row"], axis="columns")
    .groupby(["cell_id", "plate_name"])
    .transform(scale)
)

normalized_label_df = pd.concat(
    [
        label_df.loc[:, ["cell_id", "guide", "plate_name", "well_col", "well_row"]],
        normalized_label_df
    ],
    axis="columns"
)

print(normalized_label_df.shape)
normalized_label_df.head(2)

(2302, 75)


,cell_id,guide,plate_name,well_col,well_row,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,ES2,AKT1-1,Plate 1,3,A,0.655229,-0.565658,-0.839186,-0.513748,0.313600,...,0.281397,-0.279051,-0.429141,-0.177258,-0.920300,-0.139875,-0.016549,0.140570,NaN,NaN
1,ES2,AKT1-1,Plate 1,22,O,-0.251336,-0.816445,-0.525940,-0.819810,-0.450799,...,0.543716,-0.221588,-0.311041,-0.149198,-1.070176,-0.046783,0.268559,0.040163,-0.29248,0.008339


In [6]:
# Write to file
file = os.path.join("data", "labels", "normalized_cell_health_labels.tsv")
normalized_label_df.to_csv(file, index=False, sep='\t')

## Build Cell Health Target Variable GCT Files

For viewing heatmaps in Morpheus.

In [7]:
# Recode metadata variables
normalized_label_df = (
    normalized_label_df
    .rename(
        {
            "cell_id": "Metadata_cell_id",
            "guide": "Metadata_guide",
            "plate_name": "Metadata_plate_name",
            "well_col": "Metadata_well_col",
            "well_row": "Metadata_well_row"
        },
        axis="columns"
    )
)

print(normalized_label_df.shape)
normalized_label_df.head(2)

(2302, 75)


,Metadata_cell_id,Metadata_guide,Metadata_plate_name,Metadata_well_col,Metadata_well_row,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,ES2,AKT1-1,Plate 1,3,A,0.655229,-0.565658,-0.839186,-0.513748,0.313600,...,0.281397,-0.279051,-0.429141,-0.177258,-0.920300,-0.139875,-0.016549,0.140570,NaN,NaN
1,ES2,AKT1-1,Plate 1,22,O,-0.251336,-0.816445,-0.525940,-0.819810,-0.450799,...,0.543716,-0.221588,-0.311041,-0.149198,-1.070176,-0.046783,0.268559,0.040163,-0.29248,0.008339


In [8]:
# Load feature map
file = os.path.join("data", "labels", "feature_mapping_annotated.csv")
feature_map = (
    pd.read_csv(file, index_col=1)
    .transpose()
    .reset_index()
    .transpose()
    .rename({"index": "id"}, axis="rows")
)

feature_map.head()

,0,1,2,3,4,5
updated_name,,,,,,
id,original_name,feature_type,measurement,maria_thumbs_up,assay,description
well_row,Row,metadata,NaN,NaN,NaN,NaN
well_col,Column,metadata,NaN,NaN,NaN,NaN
plate_name,Plate Name,metadata,NaN,NaN,NaN,NaN
guide,sgRNA,metadata,NaN,NaN,NaN,NaN


In [9]:
# Build and output gct file
cell_health_features = [x for x in normalized_label_df.columns if not x.startswith("Metadata_")]
output_file = os.path.join("data", "labels", "normalized_cell_health_labels.gct")

write_gct(profiles=normalized_label_df,
          output_file=output_file,
          features=cell_health_features,
          meta_features="infer",
          feature_metadata=feature_map)